In [1]:
import sys
import os

sys.path.append('../src-py/')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_colwidth', None)

In [3]:
# sentence similarity
import spacy
nlp=spacy.load("en_core_web_lg")

In [4]:
from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
from transformers import BertForSequenceClassification, AutoTokenizer, TextClassificationPipeline
import torch
import json

In [5]:
from claim_scoring import *

### This notebook is to train and evalaute ArgLexRank aglgorithm:

- Paper: https://webis.de/downloads/publications/papers/alshomary_2020b.pdf

#### We will test different versions of argumentation scorers:
- Claim identification (https://aclanthology.org/N19-1054.pdf) (DONE)
- Claim Lexicon (https://github.com/hjshah142/SnippetGeneration/blob/master/data/ClaimLexicon.txt) Count number of words that appear in the lexicon and consider it the score of the sentence.
- Argumentation quality assessment Gertz et al. (https://arxiv.org/pdf/1911.11408.pdf). Code at (/mnt/ceph/storage/data-tmp/2021/jrieskamp/jl-workspace/bert-finetuning)

#### For each version, we will evaluate the top selected two sentences against the ground-truth of the following dataset: https://github.com/webis-de/sigir20-extractive-snippet-generation-for-arguments/blob/master/data/dataset_as_json_file.json

In [6]:
json_data = json.load(open('../data/dataset_as_json_file.json'))

In [7]:
len(json_data)
df=pd.DataFrame(json_data)
df[:3]

,topic,query,arg_id,sentences,snippet
0,Abortion,abortion,abortion-01,"[In 2011 there were about 730,322 abortions reported to the centers for disease control., There are about 1.7% of abortion of women's ages from 15-44 each year., Women who already had abortion earlier in there life time have abortion again., At the age of 45 a women will have at least one abortion., By the 12th week of pregnancies 88.7% of women have abortion., In the U.S. black women are 3.3 times likely to have an abortion than white women.]","[Women who already had abortion earlier in there life time have abortion again., At the age of 45 a women will have at least one abortion.]"
1,Abortion,abortion,abortion-02,"[Abortion is wrong!, Abortion Is gross!, Abortion is MURDER!!!!]","[Abortion is wrong!, Abortion is MURDER!!!!]"
2,Abortion,abortion,abortion-03,"[""The greatest destroyer of peace is abortion because if a mother can kill her own child, what is left for me to kill you and you to kill me?, There is nothing between,"" says Mother Teresa., Abortion, what does this really mean?, The definition of abortion is the deliberate termination of a human pregnancy, most often performed during the first 28 weeks., About 42 million women in the entire world with unintended pregnancies choose abortion., Abortion is a big controversial topic, people believe abortion should be illegal and others think it should stay legal., Why would someone want to ki...","[""The greatest destroyer of peace is abortion because if a mother can kill her own child, what is left for me to kill you and you to kill me?, Abortion should be illegal because abortions are not safe, laws are protecting unborn babies, and fetuses can feel pain.]"


In [8]:
#Loading transformer-based model for the embedding
model = SentenceTransformer('bert-base-uncased') #roberta-large??

Some weights of the model checkpoint at /mnt/ceph/storage/data-tmp/2021//akshitbhatia/.cache/torch/sentence_transformers/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
#Loading the claim-identification pre-trained model
claim_miner_model_path = '/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/claim-mining/export.pkl'
claim_miner  = ClaimMiner(claim_miner_model_path, 'Prediction')

In [12]:
df_results=pd.read_pickle('./results/claim_score.pkl')
df_results[:2]

,snippet,lex_p60,lex_p80,argQ_qual_170_p80,argQ_qual_170_p40,lex_p20,argQ_qual_1_p20,argQ_qual_150_p60,claim_qual_150_p60,claim_qual_170_p60,claim_qual_150_p80,claim_qual_150_p40,claim_qual_150_p20
0,"[Women who already had abortion earlier in there life time have abortion again., At the age of 45 a women will have at least one abortion.]","[(In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.5110230292699702), (By the 12th week of pregnancies 88.7% of women have abortion., 0.10474624748631271), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.10020590860624724), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.09880438168053289), (At the age of 45 a women will have at least one abortion., 0.0956903921688457), (Women who already had abortion earlier in there life time have abortion again., 0.08953004078809133)]","[(In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.34140820821284584), (By the 12th week of pregnancies 88.7% of women have abortion., 0.13992793744647422), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.13569802723291008), (At the age of 45 a women will have at least one abortion., 0.13017343640597046), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.1299542259029675), (Women who already had abortion earlier in there life time have abortion again., 0.12283816479883197)]","[(By the 12th week of pregnancies 88.7% of women have abortion., 0.17471666478108955), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.1705231739166898), (In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.1689721003697467), (At the age of 45 a women will have at least one abortion., 0.16493817286838613), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.16425485247622496), (Women who already had abortion earlier in there life time have abortion again., 0.15659503558786284)]","[(By the 12th week of pregnancies 88.7% of women have abortion., 0.1737131095274007), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.17230689713447214), (In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.17023260545087807), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.16707436713479606), (At the age of 45 a women will have at least one abortion., 0.16282560496383677), (Women who already had abortion earlier in there life time have abortion again., 0.15384741578861616)]","[(In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.8401878309959993), (By the 12th week of pregnancies 88.7% of women have abortion., 0.03476812999311297), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.033716957454896196), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.03240905608290809), (At the age of 45 a women will have at least one abortion., 0.030683234527909796), (Women who already had abortion earlier in there life time have abortion again., 0.02823479094517367)]","[(In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.1761285473876307), (By the 12th week of pregnancies 88.7% of women have abortion., 0.17321036321437078), (In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.17077794897064674), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.1654259731214429), (At the age of 45 a women will have at least one abortion., 0.1618603208622407), (Women who already had abortion earlier in there life time have abortion again., 0.15259684644366817)]","[(By the 12th week of pregnancies 88.7% of women have abortion., 0.174215454090748), (In the U.S. black women are 3.3 times likely to hav

In [11]:
#Applying ArgLexRank to generate the gist of an argument

#1. Compute sbert embedding for each sentence in the post
#2. Compute argumentativeness scores for each sentence in the post
#3. For each post, run PageRank algorithm on the graph constructed from sentences of the post (or from the post and all its comments)
#4. Take top two sentences to be the conclusion

from nltk.tokenize import word_tokenize
def get_sentences_lexicon_scores(sentences):
    #Claim lexicon model
    # read the lexicon file located in ../data/
    lexicon_file = open('../data/ClaimLexicon.txt')
    lexicon_contents = lexicon_file.read()
    lexicon_tokens = lexicon_contents.split(', ')
    score_sentences = []
    for sentence in sentences:
        score = 0  # number of times a unique word from a sentence also in lexicon
        words_counted = []  # list of words that are already counted in the sentence.
        words = word_tokenize(sentence)
        for word in words:
            
            if lexicon_tokens.__contains__(word) and not words_counted.__contains__(word):
                words_counted.append(word)
                score += 1
        score_sentences.append(score)  # append the score of the current sentence into the list.  
        
    return score_sentences

def get_sentences_argumentative_scores(sentences):
    #Gretz et al 2019 model
    # set the cuda device
    if torch.cuda.device_count() >= 1:
        DEVICE = torch.cuda.current_device()
#         log.info(torch.cuda.current_device())
#         log.info(torch.cuda.get_device_name(torch.cuda.current_device()))
    else:
        DEVICE = -1

    # set the model with trained data
    model = BertForSequenceClassification.from_pretrained('../../bert-finetuning/bert-finetuning/abhatia/results/argQ-bert-base-uncased', local_files_only=True,
                                                          cache_dir='cache')
    # set the tokenizer
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', cache_dir='cache')
    
    # set the pipeline
    pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, framework='pt', task='ArgQ')
    
    results = pipeline(sentences, device=DEVICE)
    sentences_scores = []
    for result_dict in results:
        sentences_scores.append(result_dict['score'])
#     print('type of the results are: ', type(sentences_scores))
#     print('result[0]: ', sentences_scores[0])
    return sentences_scores

def get_arg_scores(sentences):
    claim_scores = claim_miner.predict(sentences)
    return [x[1].item() for x in claim_scores]
    #return [1] * len(sentences)

def gen_match_matrix(model, sents, min_match_score=0):
    sents_embeddings  = model.encode(sents)
    
    sim_matrix = cosine_similarity(sents_embeddings, sents_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0

    return sim_matrix

def extract_gist(model, post, comments = [], min_sim_score=0, min_arg_quality_score=0, p=0.8, argumentative_scorer='claim_identifier'):
    
    all_sentences = post + comments
    #compute argumentativeness scores and filter out sentences with quality less htan arg_quality_score
    if argumentative_scorer == 'claim_identifier':
        all_sentences_arg_scores = get_arg_scores(all_sentences)
    elif argumentative_scorer == 'argQ':
        all_sentences_arg_scores = get_sentences_argumentative_scores(all_sentences)
    else:
        # represented by "lexicon_score"
        all_sentences_arg_scores = get_sentences_lexicon_scores(all_sentences)
    
    
    filtered_sentences = [x for x in zip(all_sentences, all_sentences_arg_scores) if x[1] >= min_arg_quality_score]
    
    if len(filtered_sentences) > 0:
        
        filtered_sentences_texts, filtered_sentences_arg_scores = zip(*filtered_sentences)

        sim_matrix = gen_match_matrix(model, filtered_sentences_texts)
        pr=pagerank(sim_matrix, personalize=np.array(filtered_sentences_arg_scores), p=p)

        ranked_candidates = list(zip(filtered_sentences_texts, pr))
        
    else:
        return(['failed'],0)
        
        
    return sorted(ranked_candidates, key=lambda x: -x[1])

In [50]:
# # dont run this cell

df_results['claim_qual_120_p100']=df['sentences'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.120, p=1, argumentative_scorer = 'claim_identifier')) 
df_results['claim_qual_120_p60']=df['sentences'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.120, p=0.6, argumentative_scorer = 'claim_identifier')) 
df_results['claim_qual_120_p80']=df['sentences'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.120, p=0.8, argumentative_scorer = 'claim_identifier')) 
df_results['claim_qual_120_p40']=df['sentences'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.120, p=0.4, argumentative_scorer = 'claim_identifier')) 
df_results['claim_qual_120_p20']=df['sentences'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.120, p=0.2, argumentative_scorer = 'claim_identifier')) 
df_results['claim_qual_120_p0']=df['sentences'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.120, p=0, argumentative_scorer = 'claim_identifier')) 

df_results.to_pickle('./results/claim_score.pkl')



/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


...

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)


In [51]:
df_results

,snippet,lex_p60,lex_p80,argQ_qual_170_p80,argQ_qual_170_p40,lex_p20,argQ_qual_1_p20,argQ_qual_150_p60,claim_qual_150_p60,claim_qual_170_p60,claim_qual_150_p80,claim_qual_150_p40,claim_qual_150_p20,claim_qual_120_p100,claim_qual_120_p60,claim_qual_120_p80,claim_qual_120_p40,claim_qual_120_p20,claim_qual_120_p0
0,"[Women who already had abortion earlier in there life time have abortion again., At the age of 45 a women will have at least one abortion.]","[(In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.5110230292699702), (By the 12th week of pregnancies 88.7% of women have abortion., 0.10474624748631271), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.10020590860624724), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.09880438168053289), (At the age of 45 a women will have at least one abortion., 0.0956903921688457), (Women who already had abortion earlier in there life time have abortion again., 0.08953004078809133)]","[(In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.34140820821284584), (By the 12th week of pregnancies 88.7% of women have abortion., 0.13992793744647422), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.13569802723291008), (At the age of 45 a women will have at least one abortion., 0.13017343640597046), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.1299542259029675), (Women who already had abortion earlier in there life time have abortion again., 0.12283816479883197)]","[(By the 12th week of pregnancies 88.7% of women have abortion., 0.17471666478108955), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.1705231739166898), (In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.1689721003697467), (At the age of 45 a women will have at least one abortion., 0.16493817286838613), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.16425485247622496), (Women who already had abortion earlier in there life time have abortion again., 0.15659503558786284)]","[(By the 12th week of pregnancies 88.7% of women have abortion., 0.1737131095274007), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.17230689713447214), (In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.17023260545087807), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.16707436713479606), (At the age of 45 a women will have at least one abortion., 0.16282560496383677), (Women who already had abortion earlier in there life time have abortion again., 0.15384741578861616)]","[(In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.8401878309959993), (By the 12th week of pregnancies 88.7% of women have abortion., 0.03476812999311297), (In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.033716957454896196), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.03240905608290809), (At the age of 45 a women will have at least one abortion., 0.030683234527909796), (Women who already had abortion earlier in there life time have abortion again., 0.02823479094517367)]","[(In 2011 there were about 730,322 abortions reported to the centers for disease control., 0.1761285473876307), (By the 12th week of pregnancies 88.7% of women have abortion., 0.17321036321437078), (In the U.S. black women are 3.3 times likely to have an abortion than white women., 0.17077794897064674), (There are about 1.7% of abortion of women's ages from 15-44 each year., 0.1654259731214429), (At the age of 45 a women will have at least one abortion., 0.1618603208622407), (Women who already had abortion earlier in there life time have abortion again., 0.15259684644366817)]","[(By the 12th week of p

In [54]:
df_results['argQ_qual_150_p20']=df['sentences'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.150, p=0.2, argumentative_scorer = 'argQ')) 
df_results['argQ_qual_170_p20']=df['sentences'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.170, p=0.2, argumentative_scorer = 'argQ')) 
df_results.to_pickle('./results/argQ_claim_score.pkl')

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

### load the similarity results

In [17]:
import pandas as pd
df_results = pd.read_pickle('./results/claim_score.pkl')

In [18]:
df_results.columns

Index(['snippet', 'lex_p60', 'lex_p80', 'argQ_qual_170_p80',
       'argQ_qual_170_p40', 'lex_p20', 'argQ_qual_1_p20', 'argQ_qual_150_p60',
       'claim_qual_150_p60', 'claim_qual_170_p60', 'claim_qual_150_p80',
       'claim_qual_150_p40', 'claim_qual_150_p20'],
      dtype='object')

In [19]:
for results in df_results.itertuples():
    print('results[0]: ',results[0])
    print('results[1]: ', results[1])
    print('results[2]: ', results[2])
    break
        

results[0]:  0
results[1]:  ['Women who already had abortion earlier in there life time have abortion again.', 'At the age of 45 a women will have at least one abortion.']
results[2]:  [('In the U.S. black women are 3.3 times likely to have an abortion than white women.', 0.5110230292699702), ('By the 12th week of pregnancies 88.7% of women have abortion.', 0.10474624748631271), ("There are about 1.7% of abortion of women's ages from 15-44 each year.", 0.10020590860624724), ('In 2011 there were about 730,322 abortions reported to the centers for disease control.', 0.09880438168053289), ('At the age of 45 a women will have at least one abortion.', 0.0956903921688457), ('Women who already had abortion earlier in there life time have abortion again.', 0.08953004078809133)]


In [55]:
# import spacy
# nlp=spacy.load('en_core_web_lg')

sim_results = {}
col = df_results.columns


for column in col:
    if column=='snippet':
        continue
    sim_results[column]=0
    
for results in df_results.itertuples():
    
    if len(results[1]) <=1:
        continue
    # embedding of snippets
    snippet1=nlp(results[1][0])
    snippet2=nlp(results[1][1])
    
    # embeddings of every col
    for index, column in enumerate(results):
#     for index, column in enumerate(col):
        if index==0 or index==1:  # index 0 of col is snippet
            continue
        column_name = col[index-1]
        text1 = nlp(column[0][0])
        if len(column) > 1:
            text2 = nlp(column[1][0])
#         print(index)
#         text1=nlp(results[index][0][0])  
#         text2=nlp(results[index][1][0])
        
        if snippet1.similarity(text1) ==1 or snippet1.similarity(text2)==1: 
            sim_results[column_name] += 1
        if snippet2.similarity(text1) ==1 or snippet2.similarity(text2)==1: 
            sim_results[column_name] += 1

sim_results

{'lex_p60': 50,
 'lex_p80': 50,
 'argQ_qual_170_p80': 51,
 'argQ_qual_170_p40': 57,
 'lex_p20': 52,
 'argQ_qual_1_p20': 58,
 'argQ_qual_150_p60': 56,
 'claim_qual_150_p60': 51,
 'claim_qual_170_p60': 51,
 'claim_qual_150_p80': 54,
 'claim_qual_150_p40': 52,
 'claim_qual_150_p20': 51,
 'claim_qual_120_p100': 52,
 'claim_qual_120_p60': 51,
 'claim_qual_120_p80': 54,
 'claim_qual_120_p40': 52,
 'claim_qual_120_p20': 51,
 'claim_qual_120_p0': 51,
 'argQ_qual_150_p20': 58,
 'argQ_qual_170_p20': 58}

### Using the best model to generate conclusion for the development dataset:

#### We selected the model: argQ with p=20 and quality score as 0.170, 0.150 or score as 0.170 and p=40..

In [53]:
valid_df = pd.read_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')
unique_valid_posts = valid_df.drop_duplicates('post_id')
unique_valid_posts.head()

,post_id,split,comment_id,title,post,n_sentences,counter,num_cand_conc,masked_premises,premises_with_conclusion,conclusion_stance,conclusion_targets,bart_conclusion,arglex_rank_conclusion
295914,t3_7uek1b,val,t1_dtjoww6,Pink is just light red and doesn't deserve it's own name.,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...",8,"[what's your opinion on scarlet?, brick?, salmon?, rose?, wine?]",1,"[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...","[i understand the need for more precise naming when you're doing something artsy., what i'm focusing on is how we describe colors in everyday life., pink is just a light shade of the color red., we already have a ton of precedence for just calling something a light or dark color., when people see a light or dark blue object, they just call it light blue or dark blue., when the see a light or dark green thing, they call it light green or dark green., for some reason, when we see something that is light red we call it pink., i don't see any rhyme or reason to this distinction and i don't thi...",-0.846022,Pink,i don't think pink deserves it's own name.,i don't see any rhyme or reason to this distinction and i don't think pink deserves it's name.
295939,t3_7uj99e,val,t1_dtknvk0,Elon Musk should not be selling flamethrowers as if they were toys.,"[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jokes around with it, pretending to use it on the cameraman. on twitter, he jokes about it been useful during a zombie apocalypse., but, as far as i know, no real use for it is seriously discussed., i find this behaviour borderline negligent., if these things become even more popular, it is just a matter of time until someone gets seriously hurt., either on purpo...",14,"[people get hurt with tide pods and knives, so why would we care about this?, we can't protect every dumb person in this planet from themselves.]",1,"[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jokes around with it, pretending to use it on the cameraman. on twitter, he jokes about it been useful during a zombie apocalypse., but, as far as i know, no real use for it is seriously discussed., i find this behaviour borderline negligent., if these things become even more popular, it is just a matter of time until someone gets seriously hurt., either on purpo...","[i just saw on the news that elon musk have sold over 20 thousand flamethrowers., they appear to be oversized blowtorches designed to be just below the limits that would require licenses in most states., in a promotional video, he jok

In [ ]:
unique_valid_posts['arglex_rank_conclusion'] = unique_valid_posts['masked_premises'].apply(lambda x : extract_gist(model, x, min_arg_quality_score=0.170, p=0.2, argumentative_scorer = 'argQ'))

/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/usr/local/lib/python3.6/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:145: SparseE

In [ ]:
# df.to_pickle('./results/argQ/p80.pkl')
df.to_pickle('./results/argQ/p20_q150.pkl')

In [ ]:
unique_valid_posts.to_pickle('./results/argQ/p20_q150_new.pkl')

In [38]:
# haven't used it.
unique_valid_posts.to_pickle('/mnt/ceph/storage/data-in-progress/data-research/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_unique_posts_with_arglexrank_conclusions.pkl')

In [ ]:
unique_valid_posts[['post_id', 'title', 'post', 'arglex_rank_conclusion']]

In [22]:
post_to_conc = pd.Series([ ranked_candidates[0][0] for ranked_candidates in  unique_valid_posts.arglex_rank_conclusion.tolist()], index=unique_valid_posts.post_id).to_dict()

In [24]:
valid_df['arglex_rank_conclusion'] = valid_df.post_id.apply(lambda x: post_to_conc[x])

In [26]:
valid_df.to_pickle('../../../data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation/reddit_data/conclusion_and_ca_generation/valid_conclusion_comp_remove_75sem_perc_with_targets.pkl')